<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-films/blob/Usunicie-nulli-z-danych/colabs/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Setup and Data Load
## (with joining tables)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark
# clone github repo
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-films
# Przeniesienie plików z BigData2022-films do katalogu nadrzędnego
!mv BigData2022-films/* .
!mv BigData2022-films/.* .
!rmdir BigData2022-films

In [ ]:
!git checkout Usunicie-nulli-z-danych

In [4]:
import os

# setup environment variables for our Spark Session to work
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.1-bin-hadoop3.2'

from lib.pyspark_startup import init, load

In [5]:
spark = init()

In [11]:
# Ładowanie danych z dysku google
df = load(spark = spark,
          path = "/content/drive/.shortcut-targets-by-id/1VcOir9FMG8LzEsUE-Q8YA79c_sV0tJwp/bigdata2022/")

# Preprocessing

## Odfiltorwanie odcinków

In [12]:
title_episode = spark.read.csv(
    "/content/drive/.shortcut-targets-by-id/1VcOir9FMG8LzEsUE-Q8YA79c_sV0tJwp/bigdata2022/title.episode.tsv.gz",
    sep="\t",
    header=True,
)

In [13]:
df = df.join(title_episode,df.id==title_episode.tconst,"leftanti")

## Konwersja

In [14]:
from pyspark.sql.functions import col

cols = ["dlugosc_produkcji_w_min", "rok_wydania_produkcji", "rok_zakonczenia_produkcji"]
for c in cols:
    df = df.withColumn(c, col(c).cast("float").alias(c))

## Uzupełnienie wartości dla filmów

In [15]:
from pyspark.sql.functions import when

for col in ["liczba_sezonow", "liczba_wszystkich_odcinkow"]:
    df = df.withColumn(
        col,
        when(
            df["rodzaj_produkcji"].isin(
                [
                    "short",
                    "movie",
                    "video",
                    "tvMovie",
                    "tvMiniSeries",
                    "tvSpecial",
                    "tvShort",
                ]
            ),
            1.0,
        ).otherwise(df[col]),
    )

In [16]:
df = df.withColumn(
    "rok_zakonczenia_produkcji",
    when(
        df["rodzaj_produkcji"].isin(
            [
                "short",
                "movie",
                "video",
                "tvMovie",
                "tvMiniSeries",
                "tvSpecial",
                "tvShort",
            ]
        ),
        df["rok_wydania_produkcji"],
    ).otherwise(df["rok_zakonczenia_produkcji"]),
)

## Usuniecie nulli

In [17]:
df = df.dropna(subset = df.columns[0:10])

In [18]:
df.show()

+---------+----------------+--------------------+-----------------+---------------------+-------------------------+-----------------------+--------------------+--------------+--------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|       id|rodzaj_produkcji|               tytul|czy_dla_doroslych|rok_wydania_produkcji|rok_zakonczenia_produkcji|dlugosc_produkcji_w_min|             gatunek|liczba_sezonow|liczba_wszystkich_odcinkow|        1|       10|        2|        3|        4|        5|        6|        7|        8|        9|
+---------+----------------+--------------------+-----------------+---------------------+-------------------------+-----------------------+--------------------+--------------+--------------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|tt0000015|           short| Autour d'une cabine|                0|               1894.0|  